In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
from eegyolk import helper_functions as hf # library useful for eeg and erp data cleaning
from eegyolk import initialization_functions #library to import data
from eegyolk import epod_helper

In [3]:
path_eventmarkers = os.path.join('../epod_data_not_pushed','not_zip', 'event_markers')
path_dataset=  os.path.join('../epod_data_not_pushed','not_zip')

In [6]:
trip_data= initialization_functions.load_dataset(path_dataset, file_extension = '.bdf')


99 EEG files loaded


In [7]:
trip_data[1]

['101a',
 '101b',
 '102a',
 '102b',
 '103a',
 '103b',
 '104a',
 '104b',
 '105a',
 '105b',
 '106a',
 '106b',
 '107a',
 '107b (deel 1+2)',
 '107b (deel 3+4)',
 '108a',
 '109a',
 '109b',
 '110a',
 '110b',
 '121a',
 '121b(1)',
 '121b(2)',
 '122a',
 '122b',
 '123a',
 '123b',
 '124a',
 '125a',
 '125b',
 '126a',
 '126b',
 '127a',
 '127b',
 '128a',
 '128b',
 '129a',
 '129b',
 '130a',
 '130b',
 '131a',
 '131b',
 '132a',
 '132b',
 '133a',
 '133b',
 '134a',
 '134b',
 '135a',
 '135b',
 '136a',
 '136b',
 '137a',
 '137b',
 '138a',
 '138b',
 '139a',
 '139b',
 '140a',
 '140b',
 '141a',
 '142a',
 '142b',
 '143a',
 '143b',
 '144a',
 '144b',
 '145a',
 '145b',
 '146a',
 '146b',
 '147a',
 '148a',
 '148b',
 '149a',
 '149b',
 '150a',
 '150b',
 '151a',
 '151b',
 '152a',
 '152b',
 '153a',
 '153b',
 '154a',
 '154b',
 '155a',
 '155b',
 '156a',
 '156b',
 '157a',
 '157b',
 '158a',
 '158b',
 '159a',
 '159b',
 '160a (2)',
 '160a',
 '160b']

In [9]:
initialization_functions.save_event_markers(path_eventmarkers, trip_data[0], trip_data[1])

Trigger channel has a non-zero initial value of 65540 (consider using initial_event=True to detect this event)
Removing orphaned offset at the beginning of the file.
2440 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 101 102 103 104 105 106]

 99  out of  99  saved.
